In [2]:
import requests
from pandas import DataFrame
from concurrent import futures




In [3]:
with open("서울교통공사_역주소 및 전화번호_20250318.csv", "r",encoding="euc-kr")as f:
    csv_list = f.readlines()

    print(csv_list[:5])

['연번,역번호,호선,역명,역전화번호,도로명주소,지번주소\n', '1,150,1,서울,02-6110-1331,서울특별시 중구 세종대로 지하2(남대문로 5가),서울특별시 중구 남대문로5가 73-6 서울역(1호선)\n', '2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하101(정동),서울특별시 중구 정동 5-5 시청역(1호선)\n', '3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하55(종로1가),서울특별시 종로구 종로1가 54 종각역(1호선)\n', '4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하129(종로3가),서울특별시 종로구 종로3가 10-5 종로3가역(1호선)\n']


In [7]:
#요청 url
url = "https://api.vworld.kr/req/address"

APT_KEY = "553E7667-B577-376B-A8C3-3ED2B63073A8"

#Query String 요청변수
params = {
    "service": "address",
    "request":"getcoord",
    "crs":"epsg:4326",
    "address":"판교로 242",
    "format": "json",
    "type":"road",
    "key":"553E7667-B577-376B-A8C3-3ED2B63073A8"}

#웹데이터 요청하기

with requests.Session()as session:
    session.headers.update({
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        })
  
    r=session.get(url,params=params)

    if r.status_code !=200:
        msg = "[%d Error]%s 에러가 발생함" %(r.status_code, r.reason)
        raise Exception(msg)
    
    print(r) 

 


    

           

<Response [200]>


In [9]:
mydict = r.json()
mydict

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getcoord',
   'time': '18(ms)'},
  'status': 'OK',
  'input': {'type': 'road', 'address': '판교로 242'},
  'refined': {'text': '경기도 성남시 분당구 판교로 242 (삼평동)',
   'structure': {'level0': '대한민국',
    'level1': '경기도',
    'level2': '성남시 분당구',
    'level3': '삼평동',
    'level4L': '판교로',
    'level4LC': '',
    'level4A': '삼평동',
    'level4AC': '4113565500',
    'level5': '242',
    'detail': ''}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '127.101313354', 'y': '37.402352535'}}}}

In [13]:
point = mydict['response']['result']["point"]
print(point['x'],point['y'])

127.101313354 37.402352535


In [17]:
def get_point(addr, type="road"):
    #요청 url
    url = "https://api.vworld.kr/req/address"

    #QueryString 요청 변수
    params = {
        "service":"address",
        "request": "getcoord",
        "crs":"epsg:4326",
        "address":addr,
        "format": "json",
        "type":type,
        "key":"553E7667-B577-376B-A8C3-3ED2B63073A8"
    }

    with requests.Session() as session:
        session.headers.update({
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        })
    
        r=session.get(url,params=params)

    if r.status_code !=200:
        msg = "[%d Error]%s 에러가 발생함" %(r.status_code, r.reason)
        raise Exception(msg)
    
    mydict = r.json()
    point = mydict['response']['result']["point"]
    return (point['x'],point['y'])

In [22]:
size = len(csv_list)
processes = []                                           #비동기 프로세스를 저장할 리스트
resultset = []                                           #비동기 작업의 결과를 저장할 빈리스트


with futures.ThreadPoolExecutor(max_workers=30)as executor:

    for i,v in enumerate(csv_list[1:]):
        print("%d/%d 진행중..." %(i+1,size))
        items = v.strip().split(",")

        pro = executor.submit(get_point, items[5])      #비동기 작업 생성
        processes.append(pro)                           #비동기 작업을 미리 준비한 리스트에 저장한다.0


    for i,p in enumerate(processes):
        try:
            #비동기 프로세스가 실행되는 동안 발생하는 예외에 대비하기 위한  try~except문
    
            lat,lon = p.result()
        except Exception as e:
            try:
                lat, lon= get_point(items[6],type="parcel")
            except Exception as e2:
                lat = None
                lon =None

    #새로운 반복문이므로 직전 반복문에서 생성한 items 변수가 유효하지 않게 된다.
    # 그러므로 items 변수를 새로 생성한다.
    items = csv_list[i],strip().split(",")
    items.append(lat)
    items.append(lon)
    resultset.append(items)

resultset
                
         

1/290 진행중...
2/290 진행중...
3/290 진행중...
4/290 진행중...
5/290 진행중...
6/290 진행중...
7/290 진행중...
8/290 진행중...
9/290 진행중...
10/290 진행중...
11/290 진행중...
12/290 진행중...
13/290 진행중...
14/290 진행중...
15/290 진행중...
16/290 진행중...
17/290 진행중...
18/290 진행중...
19/290 진행중...
20/290 진행중...
21/290 진행중...
22/290 진행중...
23/290 진행중...
24/290 진행중...
25/290 진행중...
26/290 진행중...
27/290 진행중...
28/290 진행중...
29/290 진행중...
30/290 진행중...
31/290 진행중...
32/290 진행중...
33/290 진행중...
34/290 진행중...
35/290 진행중...
36/290 진행중...
37/290 진행중...
38/290 진행중...
39/290 진행중...
40/290 진행중...
41/290 진행중...
42/290 진행중...
43/290 진행중...
44/290 진행중...
45/290 진행중...
46/290 진행중...
47/290 진행중...
48/290 진행중...
49/290 진행중...
50/290 진행중...
51/290 진행중...
52/290 진행중...
53/290 진행중...
54/290 진행중...
55/290 진행중...
56/290 진행중...
57/290 진행중...
58/290 진행중...
59/290 진행중...
60/290 진행중...
61/290 진행중...
62/290 진행중...
63/290 진행중...
64/290 진행중...
65/290 진행중...
66/290 진행중...
67/290 진행중...
68/290 진행중...
69/290 진행중...
70/290 진행중...
71/290 진행중...
72/290 진행중...
7

NameError: name 'strip' is not defined